In [1]:
from waylay import WaylayClient, RestResponseError
waylay_client = WaylayClient.from_profile('demo')

# Waylay Python SDK - Custom Reponse Handling

The Waylay Python SDK exposes a number of _REST Services_ (`WaylayConfig.analytics`, `WaylayConfig.byoml`), each containing a number of _REST Resources_ that have _action methods_.

In most cases these _action methods_ directly correspond to an underlying REST endpoint of the waylay platform. 

But they simplify the access to this API by applying defaults and converting results to a representation that we deem relevant for most users.

When these choices conflict with your workflow, the following advanced features for the _action methods_ allow you to customise response handling:


### return the http response information

Most _action methods_ support a `raw=True` parameter. The unparsed result and http response information is returned in a _Response_ object with attributes
 * `body` : the result data (json and csv data is parsed to python data structures)
 * standard http information such as `url`, `method`, `headers`, `status_code`, `client_response`

In [7]:
waylay_client.analytics.query.get('151CF-temperature', raw=True)

Response(url='https://ts-analytics.waylay.io/config/query/151CF-temperature?api_version=0.19', method='GET', body={'messages': [], 'name': '151CF-temperature', 'query': {'data': [{}, {'metric': 'avgSnr'}, {'resource': '151D8'}, {'metric': 'avgSnr', 'resource': '151D8'}], 'freq': 'P1D', 'metric': 'temperature', 'resource': '151CF', 'aggregation': 'median', 'interpolation': {'method': 'pad'}}, 'meta': {'tags': ['demo'], 'description': 'demo query configuration'}, 'attrs': {'created': '2020-08-29T09:01:34.566657+00:00', 'created_by': 'users/dcf8612b-94fa-4cd4-85fb-e66a1724712a', 'modified': '2020-08-29T09:03:12.225733+00:00', 'modified_by': 'users/dcf8612b-94fa-4cd4-85fb-e66a1724712a'}, '_links': {'self': {'href': 'https://ts-analytics.waylay.io/config/query/151CF-temperature'}}}, headers=Headers({'server': 'nginx/1.17.10', 'date': 'Tue, 16 Feb 2021 14:34:37 GMT', 'content-type': 'application/json', 'content-length': '631', 'server-timing': 'config; dur=14.694690704345703; env=production;

### access http response information from an error
When an request is unsuccessfull, the client will raise an exception. 

These exceptions (from the `waylay.exceptions` module) are either (instances of subclasses of)
* a `RestRequestError` that indicates a problem before sending an api call to waylay (e.g. when input argument conversion fails)
* a `RestResponseError` that reports a problem from or after the api call to waylay. This exception gives you access to the underlying response (`response` attribute)
  * a `RestResponseParseError` error indicates a problem in processing a succesfull response from the waylay platform. All other `RestResponseError` will come from errors reported by the waylay platform itself (http status code above the `200` range).
  
Other errors can occur (such as standard python `ValueError`,`TypeError` or `AttributeError`) but these will normally indicate a programming error. Networking failures will normally result in a `ClientConnectionError`

In [8]:
# try to get the representation of a `query` entity that does not exist. 
# this will result in a `404 NOT FOUND` error 
try:
   waylay_client.analytics.query.get('where are you???', raw=True)
except RestResponseError as exc:
   print(exc)
   print('-----------------')
   print(exc.response)

AnalyticsActionError(404: 'operation=not_found_error'; GET 'https://ts-analytics.waylay.io/config/query/where%20are%20you?api_version=0.19')
-----------------
Response(url='https://ts-analytics.waylay.io/config/query/where%20are%20you?api_version=0.19', method='GET', body={'messages': []}, headers=Headers({'server': 'nginx/1.17.10', 'date': 'Tue, 16 Feb 2021 14:35:00 GMT', 'content-type': 'application/json', 'content-length': '16', 'server-timing': 'config; dur=10.253190994262695; env=production; method=GET; tenant=fa31ec5f-bf6d-4cdb-930f-2cdd38b53e21; domain=demo.waylay.io', 'access-control-allow-origin': '*', 'vary': 'Cookie', 'via': '1.1 google', 'alt-svc': 'clear'}), status_code=404, client_response=<Response [404 NOT FOUND]>)


## customise or replace dataframe conversions
In _action methods_ that return pandas DataFrames, you can use the `response_constructor` parameter to replace the dataframe constructor with your own method. 

Other _action methods_ that apply conversions to raw json data structures normally support this same optionial parameter.

A falsy value will let the method return a python representation of the json data payload.

In [9]:
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P20D', 'until':'2020-01-31'},
    response_constructor=False
)

[{'columns': ['timestamp',
   {'resource': '151CF', 'metric': 'temperature', 'aggregation': 'median'},
   {'resource': '151CF', 'metric': 'avgSnr', 'aggregation': 'median'},
   {'resource': '151D8', 'metric': 'temperature', 'aggregation': 'median'},
   {'resource': '151D8', 'metric': 'avgSnr', 'aggregation': 'median'}],
  'data': [[1578700800000, 17.0, 20.134999999999998, 16.0, 21.215],
   [1578787200000, 17.0, 19.950000000000003, 16.0, 21.51],
   [1578873600000, 21.0, 19.64, 20.0, 21.15],
   [1578960000000, 21.0, 19.43, 20.0, 20.92],
   [1579046400000, 21.0, 19.755000000000003, 20.0, 18.93],
   [1579132800000, 21.0, 20.619999999999997, 20.5, 20.81],
   [1579219200000, 21.0, 20.314999999999998, 20.0, 19.915],
   [1579305600000, 17.0, 20.465, 16.0, 21.845],
   [1579392000000, 17.0, 21.43, 16.0, 21.795],
   [1579478400000, 21.0, 20.65, 20.0, 21.605],
   [1579564800000, 20.0, 20.795, 20.0, 20.715],
   [1579651200000, 20.5, 20.9, 20.0, 21.560000000000002],
   [1579737600000, 21.0, 21.6, 20

In [10]:
# return a map with timestamps as keys, the observation values as value. 
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P20D', 'until':'2020-01-31'},
    response_constructor=lambda d: { row[0] : row[1:] for row in d[0]['data'] }
)

{1578700800000: [17.0, 20.134999999999998, 16.0, 21.215],
 1578787200000: [17.0, 19.950000000000003, 16.0, 21.51],
 1578873600000: [21.0, 19.64, 20.0, 21.15],
 1578960000000: [21.0, 19.43, 20.0, 20.92],
 1579046400000: [21.0, 19.755000000000003, 20.0, 18.93],
 1579132800000: [21.0, 20.619999999999997, 20.5, 20.81],
 1579219200000: [21.0, 20.314999999999998, 20.0, 19.915],
 1579305600000: [17.0, 20.465, 16.0, 21.845],
 1579392000000: [17.0, 21.43, 16.0, 21.795],
 1579478400000: [21.0, 20.65, 20.0, 21.605],
 1579564800000: [20.0, 20.795, 20.0, 20.715],
 1579651200000: [20.5, 20.9, 20.0, 21.560000000000002],
 1579737600000: [21.0, 21.6, 20.0, 20.759999999999998],
 1579824000000: [20.5, 21.314999999999998, 20.0, 21.69],
 1579910400000: [17.0, 22.895, 16.0, 23.11],
 1579996800000: [17.0, 22.12, 16.0, 21.86],
 1580083200000: [21.0, 21.19, 21.0, 21.134999999999998],
 1580169600000: [21.0, 21.29, 20.0, 20.735],
 1580256000000: [21.0, 20.94, 20.0, 20.625],
 1580342400000: [21.0, 20.655, 20.0, 2

In [11]:
import numpy as np
# return timeseries data as a numpy array, transposed as an array per series 
waylay_client.analytics.query.data(
    '151CF-temperature', 
    params={'window': 'P20D', 'until':'2020-01-31'},
    response_constructor=lambda d: np.transpose(d[0]['data'])
)

array([[1.5787008e+12, 1.5787872e+12, 1.5788736e+12, 1.5789600e+12,
        1.5790464e+12, 1.5791328e+12, 1.5792192e+12, 1.5793056e+12,
        1.5793920e+12, 1.5794784e+12, 1.5795648e+12, 1.5796512e+12,
        1.5797376e+12, 1.5798240e+12, 1.5799104e+12, 1.5799968e+12,
        1.5800832e+12, 1.5801696e+12, 1.5802560e+12, 1.5803424e+12],
       [1.7000000e+01, 1.7000000e+01, 2.1000000e+01, 2.1000000e+01,
        2.1000000e+01, 2.1000000e+01, 2.1000000e+01, 1.7000000e+01,
        1.7000000e+01, 2.1000000e+01, 2.0000000e+01, 2.0500000e+01,
        2.1000000e+01, 2.0500000e+01, 1.7000000e+01, 1.7000000e+01,
        2.1000000e+01, 2.1000000e+01, 2.1000000e+01, 2.1000000e+01],
       [2.0135000e+01, 1.9950000e+01, 1.9640000e+01, 1.9430000e+01,
        1.9755000e+01, 2.0620000e+01, 2.0315000e+01, 2.0465000e+01,
        2.1430000e+01, 2.0650000e+01, 2.0795000e+01, 2.0900000e+01,
        2.1600000e+01, 2.1315000e+01, 2.2895000e+01, 2.2120000e+01,
        2.1190000e+01, 2.1290000e+01, 2.094000